In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [11]:
# Device (GPU jeśli dostępne)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ścieżki do danych
train_path = 'data/train'
test_path = 'data/test'

# Transformacje
transform = transforms.Compose([
    transforms.Grayscale(),                  # Na wypadek, gdyby obraz był RGB
    transforms.Resize((48, 48)),             # Rozmiar jak w oryginalnym FER2013
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))     # Standaryzacja do [-1, 1]
])

# Datasety
train_dataset = datasets.ImageFolder(train_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)

# DataLoadery
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Mapowanie klas
label_map = {idx: class_name for idx, class_name in enumerate(train_dataset.classes)}
print("Mapa etykiet:", label_map)


Mapa etykiet: {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6'}


In [12]:
class EmotionCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(EmotionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(64 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))   # [B, 32, 24, 24]
        x = self.pool(F.relu(self.conv2(x)))   # [B, 64, 12, 12]
        x = x.view(x.size(0), -1)              # flatten
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

model = EmotionCNN(num_classes=len(label_map)).to(device)


In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")


KeyboardInterrupt: 

In [9]:
# Funkcja do wizualizacji kilku obrazów i ich predykcji
def visualize_predictions(model, dataloader, label_map, num_images=6):
    model.eval()  # Ustawienie modelu w tryb ewaluacji
    images, labels = next(iter(dataloader))  # Pobierz pierwszy batch z dataloadera
    images, labels = images.to(device), labels.to(device)

    # Przewidywania modelu
    with torch.no_grad():
        outputs = model(images)  # Wykonanie forward pass
        _, predicted = torch.max(outputs, 1)  # Wyciąganie przewidywanych etykiet

    # Wizualizacja
    fig, axes = plt.subplots(2, 3, figsize=(10, 7))
    axes = axes.ravel()

    for i in range(num_images):
        ax = axes[i]
        ax.imshow(images[i].cpu().squeeze(), cmap='gray')
        true_label = label_map[labels[i].item()]
        predicted_label = label_map[predicted[i].item()]
        ax.set_title(f'True: {true_label}\nPred: {predicted_label}')
        ax.axis('off')

    plt.show()

# Przykład użycia (przewidywania na batchu)
visualize_predictions(model, test_loader, label_map)


NameError: name 'model' is not defined